# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
import plotly.express as px
import datetime
from collections import defaultdict
from tqdm import tqdm
pd.options.mode.chained_assignment = None

### Functions Setup

In [3]:
def offer_mapper(value, key1, key2):
    if key1 in value:
        return value[key1]
    if key2 in value:
        return value[key2]
    
def reward_mapper(value, key1):
    if key1 in value:
        return value[key1]
    else:
        return 0

In [4]:
def id_mapper(df, col_name):
    id_dict = {}
    id_counter = 1
    for idx in range(df.shape[0]):
        if ~(df[col_name].iloc[idx] in id_dict):
            id_dict[df[col_name].iloc[idx]] = id_counter
            id_counter += 1
    return id_dict

### Review available dataframes

In [5]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [6]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [7]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


### Data Cleaning

First, we are going to rename ID columns to bring these columns to a more standard form and to avoid confusion, which id we are referring to during analysis. 

Even though string id are okay, it is more workable to store id in for of numbers. Using functioned defined above, we are going to create dictionaries that would map string ids to numerical ids.

In [8]:
#rename columns to bring the same data to the same format
portfolio.rename({'id': 'offer_id'}, axis = 1, inplace = True)
transcript.rename({'person': 'customer_id'}, axis = 1, inplace = True)
profile.rename({'id': 'customer_id'}, axis = 1, inplace = True)

In [9]:
#set-up mapping dictionaries for customer_id and offer_id
customer_dict = id_mapper(profile, 'customer_id')
offer_dict = id_mapper(portfolio, 'offer_id')

#### Portfolio Cleaning

In [10]:
#convert string ID into numeric id
portfolio['offer_id'] = portfolio['offer_id'].map(offer_dict)

#create dummy variables for channels
portfolio['email'] = portfolio['channels'].apply(lambda x: int('email' in x))
portfolio['mobile'] = portfolio['channels'].apply(lambda x: int('mobile' in x))
portfolio['social'] = portfolio['channels'].apply(lambda x: int('social' in x))
portfolio['web'] = portfolio['channels'].apply(lambda x: int('web' in x))

#drop channels column
portfolio.drop('channels', inplace = True, axis = 1)

portfolio['duration_hours'] = portfolio['duration'].apply(lambda x: x) * 24

In [282]:
profile['age'] = profile['age'].apply(lambda x: None if x == 118 else x)
profile = profile.fillna({"age": profile['age'].dropna().median()})
profile = profile.fillna({"income": profile['income'].dropna().median()})
profile = profile.fillna({"gender": 'U'})

In [283]:
profile

,gender,age,customer_id,became_member_on,income,year_joined,offer_1_completed,offer_2_completed,offer_3_completed,offer_4_completed,...,offer_10_completed,total_spend,average_spend,total_reward,average_reward,accident_completion,offers_ignored,offer_bogo_not_completed,offer_informational_not_completed,offer_discount_not_completed
0,U,55.0,1,2017-02-12,64000.0,2017,0,0,0,0,...,0,20.40,2.266667,5,2.5,0,0,0,0,3
1,F,55.0,2,2017-07-15,112000.0,2017,0,0,0,0,...,0,77.01,25.670000,0,0.0,1,1,0,0,0
2,U,55.0,3,2018-07-12,64000.0,2018,0,0,0,0,...,0,14.30,2.383333,0,0.0,0,1,1,0,0
3,F,75.0,4,2017-05-09,100000.0,2017,1,0,0,1,...,0,159.27,22.752857,15,7.5,1,0,0,0,0
4,U,55.0,5,2017-08-04,64000.0,2017,0,0,1,0,...,0,4.65,1.550000,0,0.0,0,1,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,F,45.0,16996,2018-06-04,54000.0,2018,0,0,0,0,...,0,20.03,2.861429,0,0.0,0,0,1,1,1
16996,M,61.0,16997,2018-07-13,72000.0,2018,0,0,0,0,...,0,25.97,3.710000,0,0.0,1,1,0,1,0
16997,M,49.0,16998,2017-01-26,73000.0,2017,0,0,0,0,...,0,39.74,4.967500,0,0.0,0,2,0,0,0
16998,F,83.0,16999,2016-03-07,50000.0,2016,0,1,0,0,...,0,189.67,13.547857,10,10.0,2,0,0,0,0


#### Transcript Cleaning

In [11]:
#convert string ID into numeric id
transcript['customer_id'] = transcript['customer_id'].map(customer_dict)

profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format = '%Y%m%d')
profile['year_joined'] = profile['became_member_on'].dt.year

#convert string ID into numeric id
profile['customer_id'] = profile['customer_id'].map(customer_dict)

In [23]:
# Create separate dataframes for offer status and transactions
offer_activity = transcript[transcript['event'] != 'transaction' ]
transactions = transcript[transcript['event'] == 'transaction' ]

# map offer activity with set IDs and reward amounts
offer_activity.loc[:, 'offer_id'] = offer_activity['value'].apply(offer_mapper, args=('offer id', 'offer_id',))
offer_activity.loc[:, 'reward'] = offer_activity['value'].apply(reward_mapper, args=('reward',))
offer_activity.loc[:, 'offer_id'] = offer_activity['offer_id'].map(offer_dict)

offer_activity.drop('value', inplace = True, axis = 1)

In [29]:
transactions['amount'] = transactions['value'].apply(lambda x: x['amount'])
transactions.drop('value', inplace = True, axis = 1)

We are going to count how many times each customer completed an each offer, we already have event "offer completed"; however, there are a couple of issues with how this flag operates:
- it counts as completed even if the customer has not seen that the offer existed
- it does not count if the customer made a purchase after informational offer was viewed

We need to check for every completed reward offer, if that offer was viewed, and only then count it.
If the offer is informational (we have two types of them), we need to see if the offer was viewed, and if there was any transaction during the offer duration

Only reward offers have "completed" status, when these are completed. In order to check real completion of the offer, we would get customer activity, then scroll though the activity transcript, find the offers that are completed and pull activity that relates to this offer in the timeframe of this offer. If this activity log has two logs (these would be "received" and "completed") - this offer should not really count as completed. If it has more than three logs, it means the customer had a log "viewed" - it means it should be counted.

In the same loop we are going to create a map of total rewards that the customer received "consiously". We know that some customers completed some offers accidentally, without even knowing. These people also received the reward. We would like to count only the rewards that were saught after, aka when the customer has viewed the offer and then completed it. It means that we could increment the reward total every time we validate a completed offer.

We have plenty of customers that did not complete any offer. It looks like completing an offer is a minority class big time.

Next we are going to investigate how many informational offers were completed. These offers are not marked as completed by the system, but we are going to call them a success if a customer made a purchase after opening the offer and during the duration of the offer.

In [13]:
#list of offer IDs 
offer_ids = list(portfolio['offer_id'].unique())

# list of all customer IDs
customer_ids = list(offer_activity['customer_id'].unique())

In [88]:
def generate_offer_log(customer_offer_activity,
                       act,
                       portfolio):
    '''
    INPUT
    --------
    customer_offer_activity - dataframe of customer activity by offers
    act - activity number in search for the feature
    feature - string, feature that we are looking for
    profile_df - dataframe to be modified
    portfolio_df - dataframe with information on offers
    direct = could be 1 or -1, specifies if to go forward in time to generate offer log, or backward

    
    OUTPUT
    --------
    customer_offers_completed - dictionary of dictionary for each customer containing offer id as a key and 
                                number of times the offer was completed as value
    profile_df - containing columns for every offer id and how many times each user completed
    
    ------
    Function takes profile dataframe and returns dataframe with columns representing how many times the corresponding
    offer was completed
    
    '''
    
    
    curr_time = customer_offer_activity['time'].iloc[act]
    offer_id = customer_offer_activity['offer_id'].iloc[act]
    offer_dur = portfolio[portfolio['offer_id'] == offer_id]['duration_hours'].values[0]
    offer_log = customer_offer_activity[(customer_offer_activity['time']<=curr_time)
                                        &(customer_offer_activity['time']>=(curr_time - offer_dur))
                                        &(customer_offer_activity['offer_id']==offer_id)]
    
    return offer_log, offer_id

In [89]:
def generate_offer_log_inc(customer_offer_activity,
                       act,
                       portfolio):
    '''
    INPUT
    --------
    customer_offer_activity - dataframe of customer activity by offers
    act - activity number in search for the feature
    feature - string, feature that we are looking for
    profile_df - dataframe to be modified
    portfolio_df - dataframe with information on offers
    direct = could be 1 or -1, specifies if to go forward in time to generate offer log, or backward

    
    OUTPUT
    --------
    customer_offers_completed - dictionary of dictionary for each customer containing offer id as a key and 
                                number of times the offer was completed as value
    profile_df - containing columns for every offer id and how many times each user completed
    
    ------
    Function takes profile dataframe and returns dataframe with columns representing how many times the corresponding
    offer was completed
    
    '''
    
    
    curr_time = customer_offer_activity['time'].iloc[act]
    offer_id = customer_offer_activity['offer_id'].iloc[act]
    offer_dur = portfolio[portfolio['offer_id'] == offer_id]['duration_hours'].values[0]
    offer_log = customer_offer_activity[(customer_offer_activity['time']>=curr_time)
                                        &(customer_offer_activity['time']<=(curr_time + offer_dur))
                                        &(customer_offer_activity['offer_id']==offer_id)]
    
    return offer_log, offer_id

In [39]:
def generate_trans_log(customer_offer_activity,
                       customer_id,
                       transactions,
                       act,
                       portfolio):
    '''
    INPUT
    --------
    customer_offer_activity - dataframe of customer activity by offers
    act - activity number in search for the feature
    feature - string, feature that we are looking for
    profile_df - dataframe to be modified
    portfolio_df - dataframe with information on offers
    direct = could be 1 or -1, specifies if to go forward in time to generate offer log, or backward

    
    OUTPUT
    --------
    customer_offers_completed - dictionary of dictionary for each customer containing offer id as a key and 
                                number of times the offer was completed as value
    profile_df - containing columns for every offer id and how many times each user completed
    
    ------
    Function takes profile dataframe and returns dataframe with columns representing how many times the corresponding
    offer was completed
    
    '''
    
    
    curr_time = customer_offer_activity['time'].iloc[act]
    offer_id = customer_offer_activity['offer_id'].iloc[act]
    offer_dur = portfolio[portfolio['offer_id'] == offer_id]['duration_hours'].values[0]
    trans_log = transactions[(transactions['time']<=curr_time)
                             &(transactions['time']>=(curr_time - offer_dur))
                             &(transactions['customer_id']==customer_id)]
    
    return trans_log, offer_id

In [225]:
def generate_trans_log_inc(customer_offer_activity,
                       customer_id,
                       offer_id,
                       transactions,
                       act,
                       portfolio):
    '''
    INPUT
    --------
    customer_offer_activity - dataframe of customer activity by offers
    act - activity number in search for the feature
    feature - string, feature that we are looking for
    profile_df - dataframe to be modified
    portfolio_df - dataframe with information on offers
    direct = could be 1 or -1, specifies if to go forward in time to generate offer log, or backward

    
    OUTPUT
    --------
    customer_offers_completed - dictionary of dictionary for each customer containing offer id as a key and 
                                number of times the offer was completed as value
    profile_df - containing columns for every offer id and how many times each user completed
    
    ------
    Function takes profile dataframe and returns dataframe with columns representing how many times the corresponding
    offer was completed
    
    '''
    
    
    curr_time = customer_offer_activity['time'].iloc[act]
    offer_dur = portfolio[portfolio['offer_id'] == offer_id]['duration_hours'].values[0]
    trans_log = transactions[(transactions['time']>=curr_time)
                             &(transactions['time']<=(curr_time + offer_dur))
                             &(transactions['customer_id']==customer_id)]
    
    return trans_log

In [84]:
def customer_offers_completed_col(customer_ids = customer_ids, offer_ids = offer_ids,
                                  offer_activity = offer_activity, profile = profile,
                                 portfolio = portfolio, transactions = transactions):
    
    '''
    INPUT
    --------
    customer_ids - list of customer IDs
    offer_ids - list of offer IDs
    offer_activity_df - dataframe of customer activities on offers
    profile_df - dataframe to be modified
    portfolio - dataframe with information on offers
    transactions - dataframe with information on 
    
    OUTPUT
    --------
    customer_offers_completed - dictionary of dictionary for each customer containing offer id as a key and 
                                number of times the offer was completed as value
    profile_df - containing columns for every offer id and how many times each user completed
    
    ------
    Function takes profile dataframe and returns dataframe with columns representing how many times the corresponding
    offer was completed
    
    '''
    
    #initiate a dictionary of all completed offers
    customer_offers_completed = {}
    
    for customer_id in tqdm (customer_ids, desc="Creating customer_offers_completed dictionary..."):
        #initiate a dictionary of offers for each customer
        customer_offers_completed[customer_id] = defaultdict(int)
        #pull customer activity on offers
        customer_offer_activity = offer_activity[offer_activity['customer_id'] == customer_id].sort_values('time')
        
        # for loop on customers
        for act in range(customer_offer_activity.shape[0]):
            
            #check if award offer is really complete
            if customer_offer_activity['event'].iloc[act] == 'offer completed':
                offer_log, offer_id = generate_offer_log(customer_offer_activity = customer_offer_activity,
                                                         act = act,
                                                         portfolio = portfolio)
                if offer_log.iloc[-2]['event'] == 'offer viewed':
                    customer_offers_completed[customer_id][offer_id] = customer_offers_completed[customer_id][offer_id] + 1
                    
                    
            # check if informational offer is complete
            condition = ((customer_offer_activity['event'].iloc[act] == 'offer viewed') 
                         and (customer_offer_activity['offer_id'].iloc[act] in [3, 8]))

            
            if condition:
                
                trans_log, offer_id = generate_trans_log(customer_offer_activity = customer_offer_activity,
                                                         customer_id = customer_id,
                                                         transactions = transactions,
                                                         act = act,
                                                         portfolio = portfolio)
                if trans_log.shape[0] != 0:
                    
                    customer_offers_completed[customer_id][offer_id] = customer_offers_completed[customer_id][offer_id] + 1
                    

    # initiate offer matrix with zeros
    for offer in offer_ids:
        profile['offer_{}_completed'.format(offer)] = 0

    # fill all existing offers in the matrix
    for customer_id in tqdm (customer_ids, desc="Filling the profile_df dataframe..."):

        for offer_id in customer_offers_completed[customer_id].keys():
            profile.loc[profile['customer_id'] == customer_id ,
                        'offer_{}_completed'.format(offer_id)] = customer_offers_completed[customer_id][offer_id]
    
    return profile, customer_offers_completed

profile, customer_offers_completed = customer_offers_completed_col()

Filling the profile_df dataframe...: 100%|██████████| 16994/16994 [00:14<00:00, 1151.92it/s]


There is one expected issue in working with the "total spending" and "total reward" - these number are only going to get larger over time. Since this always increasing feature could confuse the model, we are going to convert this values from totals to average per transaction.

In [55]:
def customer_total_avg_spend_col(customer_ids = customer_ids, profile = profile, transactions = transactions):
    
    '''
    INPUT
    --------
    customer_ids - list of customer IDs
    profile_df - dataframe to be modified
    transactions_df - dataframe with information on 
    
    
    OUTPUT
    --------
    customer_ - dictionary of dictionary for each customer containing offer id as a key and 
                                number of times the offer was completed as value
    profile_df - containing columns for every offer id and how many times each user completed
    
    ------
    Function takes profile dataframe and returns dataframe with columns representing how many times the corresponding
    offer was completed
    
    '''
    
    # initialize customer total and average spend dictionaries
    customer_total_spend = {}
    customer_avg_spend = {}

    for customer_id in tqdm (customer_ids, desc="Creating customer total and average spend dictionaries..."):    
        customer_total_spend[customer_id] = transactions[transactions['customer_id'] == customer_id]['amount'].sum()
        if customer_total_spend[customer_id] > 0:
            customer_avg_spend[customer_id] = customer_total_spend[customer_id] / transactions[transactions['customer_id'] == customer_id].shape[0]
        else:
            customer_avg_spend[customer_id] = 0
            
    #initialize both columns at zero
    profile['total_spend'] = 0
    profile['average_spend'] = 0
    
    for customer_id in tqdm (customer_ids, desc="Creating customer total and average spend columns..."):
        profile.loc[profile['customer_id'] == customer_id ,
                    'total_spend'] = customer_total_spend[customer_id]
        profile.loc[profile['customer_id'] == customer_id ,
                    'average_spend'] = customer_avg_spend[customer_id]
    
    return profile, customer_total_spend, customer_avg_spend

profile, customer_total_spend, customer_avg_spend = customer_total_avg_spend_col()

Creating customer total and average spend dictionaries...: 100%|██████████| 16994/16994 [00:21<00:00, 784.94it/s]
Creating customer total and average spend columns...: 100%|██████████| 16994/16994 [00:21<00:00, 808.80it/s]


In [70]:
def customer_total_avg_reward_col(customer_ids = customer_ids, offer_ids = offer_ids,
                                  offer_activity = offer_activity, profile = profile,
                                 portfolio = portfolio, transactions = transactions):
    
    '''
    INPUT
    --------
    customer_ids - list of customer IDs
    offer_ids - list of offer IDs
    offer_activity_df - dataframe of customer activities on offers
    profile_df - dataframe to be modified
    portfolio - dataframe with information on offers
    transactions - dataframe with information on 
    
    OUTPUT
    --------
    customer_offers_completed - dictionary of dictionary for each customer containing offer id as a key and 
                                number of times the offer was completed as value
    profile_df - containing columns for every offer id and how many times each user completed
    
    ------
    Function takes profile dataframe and returns dataframe with columns representing how many times the corresponding
    offer was completed
    
    '''
    
    #initiate a dictionary of total rewards per customer 
    customer_total_reward = {}
    
    for customer_id in tqdm (customer_ids, desc="Creating customer_total_reward dictionary..."):
        # initialize dictionary for each customer
        customer_total_reward[customer_id] = defaultdict(int)
        #pull offer activity for every customer
        customer_offer_activity = offer_activity[offer_activity['customer_id'] == customer_id].sort_values('time')
        
        for act in range(customer_offer_activity.shape[0]):
            
            #check if award offer is really complete
            if customer_offer_activity['event'].iloc[act] == 'offer completed':
                offer_log, offer_id = generate_offer_log(customer_offer_activity = customer_offer_activity,
                                                         act = act,
                                                         portfolio = portfolio)
                if offer_log.iloc[-2]['event'] == 'offer viewed':
                    customer_total_reward[customer_id]['reward_amount'] = customer_total_reward[customer_id]['reward_amount'] + customer_offer_activity['reward'].iloc[act]
                    customer_total_reward[customer_id]['reward_count'] = customer_total_reward[customer_id]['reward_count'] + 1
                
                    
    
    # initialize both columns at zero
    profile['total_reward'] = 0
    profile['average_reward'] = 0

    for customer_id in tqdm (customer_ids, desc="Filling total_reward and average_reward columns..."):
        profile.loc[profile['customer_id'] == customer_id ,
                    'total_reward'] = customer_total_reward[customer_id]['reward_amount']
        if profile[profile['customer_id'] == customer_id]['total_reward'].values[0] > 0:
            profile.loc[profile['customer_id'] == customer_id ,
                        'average_reward'] = customer_total_reward[customer_id]['reward_amount'] / customer_total_reward[customer_id]['reward_count']
        else:
            profile.loc[profile['customer_id'] == customer_id ,
                        'average_reward'] = 0

    return profile, customer_total_reward

profile, customer_total_reward = customer_total_avg_reward_col()

Loading...: 100%|██████████| 16994/16994 [00:26<00:00, 634.20it/s]


Not only we need to  clean up false positives for "offer completed", we also need to flag the customers that get "offer completed" without formally "completing" the offer. These are the users, who complete the offer accidentally without actually viewing the offer. It means that every time "remove" this false positive, we need to flag the user. This would create the feature "accident_completion"



In [74]:
def customer_accident_completion_col(customer_ids = customer_ids, profile = profile, transactions = transactions):
    
    '''
    INPUT
    --------
    customer_ids - list of customer IDs
    profile_df - dataframe to be modified
    transactions_df - dataframe with information on 
    
    
    OUTPUT
    --------
    customer_ - dictionary of dictionary for each customer containing offer id as a key and 
                                number of times the offer was completed as value
    profile_df - containing columns for every offer id and how many times each user completed
    
    ------
    Function takes profile dataframe and returns dataframe with columns representing how many times the corresponding
    offer was completed
    
    '''
    #initiate a dictionary of all accidentally completed offers
    customer_accident_completion = defaultdict(int)
    
    for customer_id in tqdm (customer_ids, desc="Creating accident_completion dictionary..."):
        #pull customer activity on offers
        customer_offer_activity = offer_activity[offer_activity['customer_id'] == customer_id].sort_values('time')
        
        # for loop on customers
        for act in range(customer_offer_activity.shape[0]):
            if customer_offer_activity['event'].iloc[act] == 'offer completed':
                offer_log, offer_id = generate_offer_log(customer_offer_activity = customer_offer_activity,
                                                         act = act,
                                                         portfolio = portfolio)
                if offer_log.iloc[-2]['event'] == 'offer received':
                    customer_accident_completion[customer_id] = customer_accident_completion[customer_id] + 1
    
    # initialize accident_completion column at zero
    profile['accident_completion'] = 0

    for customer_id in tqdm (customer_ids, desc="Filling accident_completion column..."):

        profile.loc[profile['customer_id'] == customer_id ,
                    'accident_completion'] = customer_accident_completion[customer_id]
        
    return profile, customer_accident_completion

profile, customer_accident_completion = customer_accident_completion_col()

Filling accident_completion column...: 100%|██████████| 16994/16994 [00:08<00:00, 2061.83it/s]


We also have plenty of users who ignored some offers. Here are two types of ignoring users:
    
    1.users that ignored the app completely, and did not view the offer
    2.users who view the offer, but did not complete it.

The type (1) would be the "ignored_offer" - count of recieved offers, that were not viewed
The type (2) would be the "offer_not_complete" - count of viewed offers, that were not completed.

For the type (1) type of offer does not matter, since the customer did not even see the offer, they have no way of knowing what they are missing out on. For the type (2), kind of offer is important, because the offer was viewed, it means the person could estimate whether or not they wanted to pursue it. We could create a count for every type of columns (which account for duration difference), or we could count which type (bogo, discount, informational) were ignored.



Next we are going to create a column describing how many offers each customer ignored. To ignore the offer, it means the offer was received, but not viewed or completed.

The way predictions are going to work, row for every offer with the customer demographics attached

In [124]:
def customer_offers_ignored_col(customer_ids = customer_ids, profile = profile, transactions = transactions):
    
    '''
    INPUT
    --------
    customer_ids - list of customer IDs
    profile_df - dataframe to be modified
    transactions_df - dataframe with information on 
    
    
    OUTPUT
    --------
    customer_ - dictionary of dictionary for each customer containing offer id as a key and 
                                number of times the offer was completed as value
    profile_df - containing columns for every offer id and how many times each user completed
    
    ------
    Function takes profile dataframe and returns dataframe with columns representing how many times the corresponding
    offer was completed
    '''

    #initiate a dictionary of all offers
    customer_offers_ignored = defaultdict(int)


    for customer_id in tqdm (customer_ids, desc="Creating offers_ignored dictionary..."):
    # for loop on customers

        #pull offer activity for every customer
        customer_offer_activity = offer_activity[offer_activity['customer_id'] == customer_id].sort_values('time')

        for act in range(customer_offer_activity.shape[0]):

            # for loop on customer activity
            if customer_offer_activity['event'].iloc[act] == 'offer received':
                offer_log, offer_id = generate_offer_log_inc(customer_offer_activity = customer_offer_activity,
                                                                     act = act,
                                                                     portfolio = portfolio)
                if offer_log.shape[0] == 1:
                    # if the offer log only has one activity - offer received. it means the offer was ignored
                    customer_offers_ignored[customer_id] = customer_offers_ignored[customer_id] + 1
                    
                    
    # initialize accident_completion column at zero
    profile['offers_ignored'] = 0

    for customer_id in tqdm (customer_ids, desc="Filling offers_ignored column..."):

        profile.loc[profile['customer_id'] == customer_id ,
                    'offers_ignored'] = customer_offers_ignored[customer_id]
        

    return profile, customer_offers_ignored

profile, customer_offers_ignored = customer_offers_ignored_col()

Filling offers_ignored column...: 100%|██████████| 16994/16994 [00:08<00:00, 2061.21it/s]


In [126]:
profile

,gender,age,customer_id,became_member_on,income,year_joined,offer_1_completed,offer_2_completed,offer_3_completed,offer_4_completed,...,offer_7_completed,offer_8_completed,offer_9_completed,offer_10_completed,total_spend,average_spend,total_reward,average_reward,accident_completion,offers_ignored
0,None,118,1,2017-02-12,NaN,2017,0,0,0,0,...,1,0,0,0,20.40,2.266667,5,2.5,0,0
1,F,55,2,2017-07-15,112000.0,2017,0,0,0,0,...,0,0,0,0,77.01,25.670000,0,0.0,1,1
2,None,118,3,2018-07-12,NaN,2018,0,0,0,0,...,0,1,0,0,14.30,2.383333,0,0.0,0,1
3,F,75,4,2017-05-09,100000.0,2017,1,0,0,1,...,0,1,0,0,159.27,22.752857,15,7.5,1,0
4,None,118,5,2017-08-04,NaN,2017,0,0,1,0,...,0,0,0,0,4.65,1.550000,0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,F,45,16996,2018-06-04,54000.0,2018,0,0,0,0,...,0,0,0,0,20.03,2.861429,0,0.0,0,0
16996,M,61,16997,2018-07-13,72000.0,2018,0,0,0,0,...,0,0,0,0,25.97,3.710000,0,0.0,1,1
16997,M,49,16998,2017-01-26,73000.0,2017,0,0,0,0,...,0,1,0,0,39.74,4.967500,0,0.0,0,2
16998,F,83,16999,2016-03-07,50000.0,2016,0,1,0,0,...,0,0,0,0,189.67,13.547857,10,10.0,2,0


Next we are going to create a set of features that would describe how many of each type of offers each customer not completed. To not complete an offer means to receive the offer, view it, and not pursue it.

First, there are three types of offers, discount and bogo, informational offers would again be a separate case. Each customer would be assigned a dictionary with offer_type as a key and count of times not completed as values.

In [261]:
def customer_offers_not_completed_col(customer_ids = customer_ids, profile = profile, transactions = transactions,
                                     portfolio = portfolio):
    
    '''
    INPUT
    --------
    customer_ids - list of customer IDs
    profile_df - dataframe to be modified
    transactions_df - dataframe with information on 
    
    
    OUTPUT
    --------
    customer_ - dictionary of dictionary for each customer containing offer id as a key and 
                                number of times the offer was completed as value
    profile_df - containing columns for every offer id and how many times each user completed
    
    ------
    Function takes profile dataframe and returns dataframe with columns representing how many times the corresponding
    offer was completed
    '''  
    # initialize dictionary for 
    customer_offers_not_completed = {}


    # bogo_ids = portfolio[portfolio['offer_type'] == 'bogo']['offer_id'].to_list()
    # discount_ids = portfolio[portfolio['offer_type'] == 'discount']['offer_id'].to_list()
    info_ids = portfolio[portfolio['offer_type'] == 'informational']['offer_id'].to_list()
    rewards_ids = portfolio[(portfolio['offer_type'] == 'bogo')|(portfolio['offer_type'] == 'discount')]['offer_id'].to_list()
    offer_id_map = {}
    for offer_id in offer_ids:
        offer_id_map[offer_id] = portfolio[portfolio['offer_id'] == offer_id]['offer_type'].values[0]
    
    
    for customer_id in tqdm (customer_ids, desc="Creating customer_offers_not_completed dictionary..."):

        
        # initialize a dictionary for each customer
        customer_offers_not_completed[customer_id] = defaultdict(int)

        customer_offer_activity = offer_activity[offer_activity['customer_id'] == customer_id].sort_values('time')
        customer_offer_activity

        for act in range(customer_offer_activity.shape[0]):

            # for loop on customer activity
            if customer_offer_activity['event'].iloc[act] == 'offer received':
                offer_log, offer_id = generate_offer_log_inc(customer_offer_activity = customer_offer_activity,
                                                                     act = act,
                                                                     portfolio = portfolio)
                if (offer_id in rewards_ids)&(offer_log.shape[0]>1):
                    if (offer_log.shape[0]==2) & (offer_log['event'].iloc[1] == 'offer viewed'):

                        customer_offers_not_completed[customer_id][offer_id_map[offer_id]] = customer_offers_not_completed[customer_id][offer_id_map[offer_id]] + 1
                    condition = ((offer_log.shape[0]>2)
                                 and (offer_log['event'].iloc[1] == 'offer viewed')
                                 and (offer_log['event'].iloc[2] != 'offer completed'))
                    if condition:
                        customer_offers_not_completed[customer_id][offer_id_map[offer_id]] = customer_offers_not_completed[customer_id][offer_id_map[offer_id]] + 1

                if (offer_id in info_ids)&(offer_log.shape[0]>1):
                    if offer_log['event'].iloc[1] == 'offer viewed':
                        trans_log = generate_trans_log_inc(customer_offer_activity,
                                                           customer_id,
                                                           offer_id,
                                                           transactions,
                                                           act,
                                                           portfolio)
                        if trans_log.shape[0] == 0:
                            customer_offers_not_completed[customer_id][offer_id_map[offer_id]] = customer_offers_not_completed[customer_id][offer_id_map[offer_id]] + 1

    # initiate offer matrix with zeros
    for offer_type in portfolio['offer_type'].unique():
        profile['offer_{}_not_completed'.format(offer_type)] = 0

    # fill all existing offers in the matrix
    for customer_id in tqdm (customer_ids, desc="Filling the profile dataframe..."):

        for offer_type in customer_offers_not_completed[customer_id].keys():
            profile.loc[profile['customer_id'] == customer_id ,
                        'offer_{}_not_completed'.format(offer_type)] = customer_offers_not_completed[customer_id][offer_type]
  

    return profile, customer_offers_not_completed

profile, customer_offers_not_completed = customer_offers_not_completed_col()


Creating customer_offers_not_completed dictionary...: 100%|██████████| 16994/16994 [02:05<00:00, 134.91it/s]
Filling the profile dataframe...: 100%|██████████| 16994/16994 [00:07<00:00, 2179.87it/s]


In [284]:
profile

,gender,age,customer_id,became_member_on,income,year_joined,offer_1_completed,offer_2_completed,offer_3_completed,offer_4_completed,...,offer_10_completed,total_spend,average_spend,total_reward,average_reward,accident_completion,offers_ignored,offer_bogo_not_completed,offer_informational_not_completed,offer_discount_not_completed
0,U,55.0,1,2017-02-12,64000.0,2017,0,0,0,0,...,0,20.40,2.266667,5,2.5,0,0,0,0,3
1,F,55.0,2,2017-07-15,112000.0,2017,0,0,0,0,...,0,77.01,25.670000,0,0.0,1,1,0,0,0
2,U,55.0,3,2018-07-12,64000.0,2018,0,0,0,0,...,0,14.30,2.383333,0,0.0,0,1,1,0,0
3,F,75.0,4,2017-05-09,100000.0,2017,1,0,0,1,...,0,159.27,22.752857,15,7.5,1,0,0,0,0
4,U,55.0,5,2017-08-04,64000.0,2017,0,0,1,0,...,0,4.65,1.550000,0,0.0,0,1,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,F,45.0,16996,2018-06-04,54000.0,2018,0,0,0,0,...,0,20.03,2.861429,0,0.0,0,0,1,1,1
16996,M,61.0,16997,2018-07-13,72000.0,2018,0,0,0,0,...,0,25.97,3.710000,0,0.0,1,1,0,1,0
16997,M,49.0,16998,2017-01-26,73000.0,2017,0,0,0,0,...,0,39.74,4.967500,0,0.0,0,2,0,0,0
16998,F,83.0,16999,2016-03-07,50000.0,2016,0,1,0,0,...,0,189.67,13.547857,10,10.0,2,0,0,0,0


In [80]:
#total reward  completed by customer 16
print("Total Reward received by customer 16: ", dict.__repr__(customer_total_reward[16])) 

#total reward  completed by customer 18
print("Total Reward received by customer 18: ", dict.__repr__(customer_total_reward[18]) )

#total reward  completed by customer 185
print("Total Reward received by customer 185: ", dict.__repr__(customer_total_reward[185]) )

Total Reward received by customer 16:  {'reward_amount': 8, 'reward_count': 3}
Total Reward received by customer 18:  {'reward_amount': 0}
Total Reward received by customer 185:  {'reward_amount': 14, 'reward_count': 3}


In [81]:
#offers completed by customer 16
print("Completed by customer 16: ", dict.__repr__(customer_offers_completed[16]) )

#offers completed by customer 18
print("Completed by customer 18: ", dict.__repr__(customer_offers_completed[18]) )

#offers completed by customer 9
print("Completed by customer 9: ", dict.__repr__(customer_offers_completed[9]) )

Completed by customer 16:  {7: 1, 6: 2}
Completed by customer 18:  {8: 1}
Completed by customer 9:  {9: 2, 10: 1, 4: 1, 7: 1}


In [82]:
#offers accidentally completed by customer 16
print("Accidentally completed by customer 16: ", customer_accident_completion[16] )

#offers accidentally completed by customer 18
print("Accidentally completed by customer 18: ", customer_accident_completion[18]) 

#offers accidentally completed by customer 44
print("Accidentally completed by customer 44: ", customer_accident_completion[44]) 

Accidentally completed by customer 16:  1
Accidentally completed by customer 18:  0
Accidentally completed by customer 44:  3


In [244]:
#offers ignored by customer 16
print("Offers ignored by customer 16: ", customer_offers_ignored[16] )

#offers ignored by customer 5
print("Offers ignored by customer 5: ", customer_offers_ignored[5]) 

#offers ignored by customer 17000
print("Offers ignored by customer 17000: ", customer_offers_ignored[17000]) 

Offers ignored by customer 16:  0
Offers ignored by customer 5:  1
Offers ignored by customer 17000:  2


In [254]:
#offers viewed but not completed by customer 18
print("Offers viewed but not completed by customer 18: ", dict.__repr__(customer_offers_not_completed[18] ))

#offers viewed but not completed by customer 5
print("Offers viewed but not completed by customer 5: ", dict.__repr__(customer_offers_not_completed[5]) )

#offers viewed but not completed by customer 17000
print("Offers viewed but not completed by customer 17000: ", dict.__repr__(customer_offers_not_completed[17000]) )

Offers viewed but not completed by customer 18:  {'discount': 2, 'bogo': 2, 'informational': 1}
Offers viewed but not completed by customer 5:  {'discount': 2, 'informational': 1}
Offers viewed but not completed by customer 17000:  {}


#### Profile Cleaning

In [285]:
profile

,gender,age,customer_id,became_member_on,income,year_joined,offer_1_completed,offer_2_completed,offer_3_completed,offer_4_completed,...,offer_10_completed,total_spend,average_spend,total_reward,average_reward,accident_completion,offers_ignored,offer_bogo_not_completed,offer_informational_not_completed,offer_discount_not_completed
0,U,55.0,1,2017-02-12,64000.0,2017,0,0,0,0,...,0,20.40,2.266667,5,2.5,0,0,0,0,3
1,F,55.0,2,2017-07-15,112000.0,2017,0,0,0,0,...,0,77.01,25.670000,0,0.0,1,1,0,0,0
2,U,55.0,3,2018-07-12,64000.0,2018,0,0,0,0,...,0,14.30,2.383333,0,0.0,0,1,1,0,0
3,F,75.0,4,2017-05-09,100000.0,2017,1,0,0,1,...,0,159.27,22.752857,15,7.5,1,0,0,0,0
4,U,55.0,5,2017-08-04,64000.0,2017,0,0,1,0,...,0,4.65,1.550000,0,0.0,0,1,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,F,45.0,16996,2018-06-04,54000.0,2018,0,0,0,0,...,0,20.03,2.861429,0,0.0,0,0,1,1,1
16996,M,61.0,16997,2018-07-13,72000.0,2018,0,0,0,0,...,0,25.97,3.710000,0,0.0,1,1,0,1,0
16997,M,49.0,16998,2017-01-26,73000.0,2017,0,0,0,0,...,0,39.74,4.967500,0,0.0,0,2,0,0,0
16998,F,83.0,16999,2016-03-07,50000.0,2016,0,1,0,0,...,0,189.67,13.547857,10,10.0,2,0,0,0,0


 Map Total Reward to the profile dataset

In [286]:
profile.columns

Index(['gender', 'age', 'customer_id', 'became_member_on', 'income',
       'year_joined', 'offer_1_completed', 'offer_2_completed',
       'offer_3_completed', 'offer_4_completed', 'offer_5_completed',
       'offer_6_completed', 'offer_7_completed', 'offer_8_completed',
       'offer_9_completed', 'offer_10_completed', 'total_spend',
       'average_spend', 'total_reward', 'average_reward',
       'accident_completion', 'offers_ignored', 'offer_bogo_not_completed',
       'offer_informational_not_completed', 'offer_discount_not_completed'],
      dtype='object')

In [298]:
demo_cols = ['gender', 'age', 'customer_id', 'income',
                           'year_joined', 'total_spend',
                           'average_spend', 'total_reward', 'average_reward',
                           'accident_completion', 'offers_ignored', 'offer_bogo_not_completed',
                           'offer_informational_not_completed', 'offer_discount_not_completed']
demographic_offer = profile[demo_cols]
demographic_offer['offer_completed'] = 0

In [299]:
demographic_offer

,gender,age,customer_id,income,year_joined,total_spend,average_spend,total_reward,average_reward,accident_completion,offers_ignored,offer_bogo_not_completed,offer_informational_not_completed,offer_discount_not_completed,offer_completed
0,U,55.0,1,64000.0,2017,20.40,2.266667,5,2.5,0,0,0,0,3,0
1,F,55.0,2,112000.0,2017,77.01,25.670000,0,0.0,1,1,0,0,0,0
2,U,55.0,3,64000.0,2018,14.30,2.383333,0,0.0,0,1,1,0,0,0
3,F,75.0,4,100000.0,2017,159.27,22.752857,15,7.5,1,0,0,0,0,0
4,U,55.0,5,64000.0,2017,4.65,1.550000,0,0.0,0,1,0,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,F,45.0,16996,54000.0,2018,20.03,2.861429,0,0.0,0,0,1,1,1,0
16996,M,61.0,16997,72000.0,2018,25.97,3.710000,0,0.0,1,1,0,1,0,0
16997,M,49.0,16998,73000.0,2017,39.74,4.967500,0,0.0,0,2,0,0,0,0
16998,F,83.0,16999,50000.0,2016,189.67,13.547857,10,10.0,2,0,0,0,0,0


In [343]:
demographic_offer_const = pd.DataFrame(columns = demographic_offer.columns)

# customer_id = 16
for customer_id in tqdm (customer_ids, desc="Creating demografic_offer dataframe..."):


    offers = []
    for offer_id in customer_offers_completed[customer_id].keys():
        for time in range(customer_offers_completed[customer_id][offer_id]):
            offers.append(offer_id)

    for offer_id in offers:
        demographic_offer_const = demographic_offer_const.append(demographic_offer[demographic_offer['customer_id']==customer_id])
        demographic_offer_const['offer_completed'].iloc[-1] = offer_id

    if len(offers)==0:
        demographic_offer_const = demographic_offer_const.append(demographic_offer[demographic_offer['customer_id']==customer_id])
    

Creating demografic_offer dataframe...: 100%|██████████| 16994/16994 [04:01<00:00, 70.46it/s]


In [344]:
demographic_offer_const

,gender,age,customer_id,income,year_joined,total_spend,average_spend,total_reward,average_reward,accident_completion,offers_ignored,offer_bogo_not_completed,offer_informational_not_completed,offer_discount_not_completed,offer_completed
3,F,75.0,4,100000.0,2017,159.27,22.752857,15,7.5,1,0,0,0,0,4
3,F,75.0,4,100000.0,2017,159.27,22.752857,15,7.5,1,0,0,0,0,8
3,F,75.0,4,100000.0,2017,159.27,22.752857,15,7.5,1,0,0,0,0,1
4,U,55.0,5,64000.0,2017,4.65,1.550000,0,0.0,0,1,0,1,2,3
5,M,68.0,6,70000.0,2018,57.73,19.243333,7,3.5,0,1,0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12537,M,44.0,12538,64000.0,2017,161.89,16.189000,0,0.0,0,0,0,0,0,8
12774,U,55.0,12775,64000.0,2015,6.25,1.041667,0,0.0,0,0,0,0,1,0
12945,M,61.0,12946,73000.0,2018,80.58,20.145000,0,0.0,1,0,0,0,0,0
14529,F,52.0,14530,86000.0,2017,164.57,23.510000,3,3.0,0,0,0,0,0,6


In [340]:
customer_offers_completed[customer_id]

defaultdict(int, {7: 1, 6: 2})

In [341]:
offers

[7, 6, 6]

### Exploratory Data Analysis

At first, let's take a look at a single customer and explore their transactions/offers. How do they correlate and follow one another?

Let's find an "average" customer who used lots of offers to evaluate the best strategy to process them. Our goal is come up with a recommendation system that would suggest the best offers to customers to maximize sales/profit

In [291]:
profile[['offer_1_completed', 'offer_2_completed',
       'offer_3_completed', 'offer_4_completed', 'offer_5_completed',
       'offer_6_completed', 'offer_7_completed', 'offer_8_completed',
       'offer_9_completed', 'offer_10_completed']].sum(axis = 1)

0        2
1        0
2        1
3        3
4        1
        ..
16995    0
16996    0
16997    1
16998    1
16999    1
Length: 17000, dtype: int64

In [ ]:
transcript['time'].max()

In [ ]:
profile

In [ ]:
profile['age'].mean()

Considering the median age of the entire US population is about [38.2 years](https://en.wikipedia.org/wiki/Demographics_of_the_United_States). Mean age of Starbucks customer to be 62 seems too high, considering that Starbucks does not really market their product to the older poulation.

In [ ]:
profile_max_age = profile[profile['age']==profile['age'].max()].sort_values('became_member_on')
profile_max_age

In [ ]:
profile_other = profile[profile['age']<profile['age'].max()].sort_values('became_member_on')
profile_other

There are 2175 customers, whose age is assigned as 118 year, and all of their information is missing, while customer with age different from 118 have data associated with their profile. This could be a sign up issue, maybe optional fields or using the app in a "guest" mode? 

In [ ]:
profile_max_age.isna().sum()/profile_max_age.shape[0]

In [ ]:
profile_other.isna().sum()

It looks like all of the people whose age is 118 are missing "gender", "income", and practically missing age (118 must be a place holder). For now we are going to treat peolple with the age 118 as "guests" or new members

In [ ]:
print("Starbucks App user age is between {} and {} years old"
      .format(profile_other['age'].min(), profile_other['age'].max()))


In [ ]:
fig = px.histogram(profile_other,
                   x="age",
                   nbins=10,
                   color_discrete_sequence=['indigo', 'indianred','pink'],
                  color="gender",
                  barmode='overlay',
                  opacity = 0.50,
                  width=600, height=400)

fig.show()

In [ ]:
fig = px.histogram(profile_other,
                   x="income",
                   nbins=10,
                   color_discrete_sequence=['indigo', 'indianred','pink'],
                  color="gender",
                  barmode='overlay',
                  opacity = 0.50,
                  width=600, height=400)

fig.show()

In [ ]:
fig = px.bar(profile_other, x='gender', y='income',opacity = 1)
fig.show()

In [ ]:
fig = px.histogram(profile,
                   x="year_joined",
                   nbins=6,
#                   barmode='overlay',
                  opacity = 0.50,
                  width=600, height=400)

fig.show()

In [ ]:
customer_id = 13
user_x = transcript[transcript['customer_id'] == customer_id].sort_values('time')
user_x

In [ ]:
old_id = 'fafdcd668e3743c1bb461111dcafc2a4'
portfolio[portfolio['offer_id'] == offer_dict[old_id]]

In [ ]:
portfolio

In [ ]:
offer_dict['fafdcd668e3743c1bb461111dcafc2a4']

In [ ]:
value1 = {'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
value2 = {'offer_id': '5a8bc65990b245e5a138643cd4eb9837'}
value3 = {'offer_id': '3f207df678b143eea3cee63160fa8bed'}
value4 = {'offer id': '3f207df678b143eea3cee63160fa8bed'}
values = [value1, value2, value3, value4]
transcript[transcript['value'].isin(values)]

In [ ]:
offer_activity[offer_activity['event'] == 'offer completed']

In [ ]:
offer_activity.loc[306509]['value']

In [ ]:
offer_activity